<a href="https://colab.research.google.com/github/wnbh/installfclone/blob/main/aria2c%E8%A7%A3%E5%8E%8B%E4%B8%8A%E4%BC%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <font size="4" color=red><b>fclone挂载云盘</b></font>
from IPython.display import clear_output
 
def choose():
  print("\n输入序号:\n1.nohup挂载   2.tmux挂载\n3.screen挂载  4.drive挂载\n5.查看已挂载   6.卸载云盘\n7.退出\n默认以tmux模式挂载\n")
  输入 = input("选择模式  ")
  return 输入
def remote():
  !fclone listremotes | sed 's/://g' | awk '{print FNR"."$0}' > /content/log.txt #列出rclone.conf中的项目|将:替换为空白|在每行前面添加行号
  !cat /content/log.txt | awk 'BEGIN{ORS=" "}{print $0}'
  输入 = input("\n输入序号挂载，回车取消  ")
  while True:
    if 输入 == "":
      print("取消挂载")
      项目 = ""
      break
    else:
      for remote in open("/content/log.txt"):
        if 输入 in remote.split(".")[0]:
          项目 = remote.split(".")[-1].strip()
          remote = "存在"
          break
    if remote == "存在":
      break
    else:
      输入 = input("请输入正确序号或者回车取消挂载  ")
  return 项目
def mkdir(): #子程序
  print("fclone挂载模式\n")
  项目 = remote()
  挂载项目 = 项目
  del 项目
  if 挂载项目:
    地址 = input("填写云盘挂载的地址  ")
    if 地址 == "":
      地址 = "/content/mydrive"
    else:
      地址 = f'/content{地址}'
    挂载地址 = f'{地址}/{挂载项目}'
    print("挂载地址:"+挂载地址)
    mkdirlambda = lambda x: os.makedirs(x) if not os.path.exists(x)  else True  # 目录是否存在,不存在则创建
    mkdirlambda(f'{地址}/{挂载项目}')
  else:
    挂载地址 = ""
  return 挂载项目,挂载地址
 
print("检查fclone")
!fclone > /content/log.txt 2>&1
with open("/content/log.txt") as fp:
  if "fclone: command not found" in fp.read():
    print("正在安装")
    !bash <(wget -qO- https://git.io/JJYE0)
    clear_output()
print("fclone已安装\n")
print("检查screen")
!dpkg -l screen  >> /content/log.txt 2>&1
with open("/content/log.txt") as fp:
  if "no packages found matching screen" in fp.read():
    print("正在安装")
    !apt-get -y install screen #安装screen以便多窗口运行
    clear_output()
del fp
print("screen已安装\n")
print("检查rclone.conf")
import os
if not os.path.isfile("/root/.config/rclone/rclone.conf"):
  !wget "https://gdshare.lqyr.workers.dev/api/download/rclone.conf?id=15WA3CA57Fynu_NuVFH09miFnM1qVEjez&sig=ed985101ff3562793bdc20c82892667462a5316c346a210e023811d342f8f70d" -O /root/.config/rclone/rclone.conf
  print("rclone.conf已配置好\n")
else:
  print("rclone.conf已配置好\n")
  print("输入序号:1.加载  2.不加载  默认不加载")
  输入 = input("是否重新加载rclone.conf  ")
  while True:
    if 输入 == "1":
      !wget "https://gdshare.lqyr.workers.dev/api/download/rclone.conf?id=15WA3CA57Fynu_NuVFH09miFnM1qVEjez&sig=ed985101ff3562793bdc20c82892667462a5316c346a210e023811d342f8f70d" -O /root/.config/rclone/rclone.conf
      break
    elif 输入 == "2" or 输入 == "":
      break
    else:
      输入 = input("只能输入1/2，请重新输入  ")
clear_output()
!fclone config show
 
输入 = choose()
while True:
  if 输入 == "1" or 输入 == "2" or 输入 == "3" or 输入 == "":
    挂载项目,挂载地址 = mkdir()
    if not 挂载地址:
      输入 = "取消挂载"
  if 输入 == "1":
    !nohup fclone mount "{挂载项目}": "{挂载地址}" --copy-links --no-gzip-encoding --no-check-certificate --allow-other --allow-non-empty --umask 000 >/content/log.txt 2>&1 &
  elif 输入 == "2" or 输入 == "":
    !tmux ls > /content/log.txt 2>&1 #列出所有窗口
    for a in open("/content/log.txt"):
      b = a.split(':')[0].strip()
      if 挂载项目 == b:
        窗口 = "存在"
        break
      else:
        窗口 = "不存在"
    if 窗口 == "不存在":
      !tmux new -s "{挂载项目}" -d
    !tmux send -t "{挂载项目}" 'fclone mount "{挂载项目}": "{挂载地址}" --copy-links --no-gzip-encoding --no-check-certificate --allow-other --allow-non-empty --umask 000' Enter
  elif 输入 == "3":
    !screen -dmS "{挂载项目}" fclone mount "{挂载项目}": "{挂载地址}" --copy-links --no-gzip-encoding --no-check-certificate --allow-other --allow-non-empty --umask 000 #新建一个窗口并挂载云盘
  elif 输入 == "4":
    print("自带挂载模式\n")
    挂载项目 = !df -Th | grep fuse.drive | awk 'BEGIN{ORS="\n"}{print""$1"|"$7""}' #BEGIN{ORS="\n"}定义输入内容各行之间的符号,grep fuse.drive查找含有fuse.drive的行
    if 挂载项目:
      print("已存在挂载点\n1.是 2.否")
      输入 = input("是否重新挂载，默认是 ")
      while True:
        if 输入 == "1" or 输入 == "":
          for a in 挂载项目:
            项目地址 = a.split("|")[-1]
          !umount "{项目地址}" #卸载网盘
          !rm -rf "{项目地址}"
          !rm -rf "/root/.config/Google"
          break
        elif 输入 == "2":
          输入 = "取消挂载"
          break
        else:
          输入 = input("只能输入1/2，请重新输入  ")
    if not 挂载项目 or 输入 == "1" or 输入 == "":
      from google.colab import drive
      print("选择挂载的账号:\n1.lmq9801  2.panbanyun  3.自定义")
      输入 = input("选择挂载账号，回车取消挂载  ")
      while True:
        if 输入 == "1":
          !wget -q "https://gdshare.lqyr.workers.dev/api/download/lmq9801.zip?id=1vXhS7nMHyATlf9RdjdIRgDBiMo_bext7&sig=8deebb8afaee4db23df98c9a55ae70e5e65c10d32fadc8be9c61c6e90060a876" -O /content/lmq9801.zip
          !7z x -aoa "/content/lmq9801.zip" -o"/root/.config" > /dev/null
          drive.mount('/content/lmq9801')
          挂载地址 = "/content/lmq9801"
          break
        elif 输入 == "2":
          !wget -q "https://gdshare.lqyr.workers.dev/api/download/panbanyun.zip?id=1ITsdPjmtMgomD0u-ah96KPwMff2zR6XX&sig=8151365bf52ad5ac251fabc7e513c1a4b91ec31a6639c93289821d5d180cc908" -O /content/panbanyun.zip > log.txt
          !7z x -aoa "/content/panbanyun.zip" -o"/root/.config" > /dev/null
          drive.mount('/content/panbanyun')
          挂载地址 = "/content/panbanyun"
          break
        elif 输入 == "3":
          挂载项目 = input("输入项目名称  ")
          挂载地址 = f'/content/{挂载项目}'
          drive.mount(挂载地址)
          break
        elif 输入 == "":
          输入 = "取消挂载"
          break
        else:
          输入 = input("只能输入1/2/3，请重新输入  ")
  elif 输入 == "5":
    clear_output()
    print("\n查看已挂载云盘\n")
    !df -h
    del 输入
    break
  elif 输入 == "6":
    try:
      卸载项目
    except NameError:
      print("卸载云盘\n")
      !df -h
      卸载项目 = input("\n填写已挂载的项目名称  ")
    !df -Th | grep fuse | awk 'BEGIN{ORS="\n"}{print""$1"|"$7""}' | sed 's/://g' > /content/log.txt #BEGIN{ORS="\n"}定义输入内容各行之间的符号,grep fuse查找含有fuse的行
    for a in open("/content/log.txt"):
      drive = a.split('|')[0].strip()
      if 卸载项目 == drive:
        项目地址 = a.split('|')[-1].strip() # 用:分割字符串，并保存后面的内容
        print("已找到，项目地址为:"+项目地址)
        !umount "{项目地址}" #卸载网盘
        !rm -rf "{项目地址}"
        if 卸载项目 == "drive":
          !rm -rf "/root/.config/Google"
        卸载项目 = "已找到"
    if 卸载项目 == "已找到":
      if os.path.isdir(项目地址):
        print("\n卸载失败")
      else:
        print("已卸载\n")
        !df -h
    else:
      print("未挂载此项目")
    卸载项目 = input("回车返回上一级或者继续卸载  ")
    if 卸载项目 == "":
      del 卸载项目
    else:
      continue
  elif 输入 == "7":
    del 输入
    clear_output()
    break
  elif 输入 == "取消挂载":
    输入 = choose()
    continue
  else:
    print("只能输入1/2/3/4/5/6/7，请重新输入")
  if 输入 == "1" or 输入 == "2" or 输入 == "3" or 输入 == "4" or 输入 == "":
    !ls {挂载地址} > /dev/null
    import time
    time.sleep(1) #休眠2秒
    if os.listdir(挂载地址):
      print("成功挂载")
    else:
      print("\n挂载可能失败")
    del 挂载地址
  del 输入
  输入 = choose()

In [ ]:
#@markdown <font size="4" color=red><b>提取挂载信息</b></font>
from IPython.display import clear_output
%cd /root/.config
!zip -v -r -u /content/Google.zip Google -x "./Google/DriveFS/*/metrics_sqlite.db-wal"
%cd /content
clear_output()
from google.colab import files
files.download('Google.zip')

In [ ]:
#@markdown <h3>文件操作</font></h3><br>
模式 = "\u5220\u9664" #@param ["复制","移动","删除"]
源地址 =  "" #@param {type:"string"}
目的地 =  "" #@param {type:"string"}
if 模式 == "复制":
  print("开始复制")
  !cp -rf "$源地址" "$目的地"
  print("已复制")
elif 模式 == "移动":
  print("开始移动")
  !mv -rf "$源地址" "$目的地"
  print("已移动")
else:
  !rm -rf "$源地址"
  print("已删除")
#@markdown <p>- 删除选项的地址即源地址

In [8]:
 
#@markdown <font size="4" color=red><b>SSH</b></font>
from IPython.display import clear_output
import urllib.request, json#, random, string, getpass
 
#Generate root password
#随机密码
#password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
password = 'LIN199896' #@param {type:"string"}
 
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
 
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
 
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
 
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')
 
#Ask token
#print("Copy authtoken from https://dashboard.ngrok.com/auth")
#键入获取token
#authtoken = getpass.getpass()
authtoken = "1q2878VMD0fPL9nqtQ8imerH3g0_5E8pYkcu8Ksv3bwGW8eok" #@param {type:"string"}
 
#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
 
#Get public address and print connect command
while True:
  try:
    with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
      data = json.loads(response.read().decode())
      (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
    break
  except Exception as link_fault:
    continue
clear_output()
 
#Print root password
print(f'SSH command: ssh root@{host}:{port}')
print(f'Root password: {password}')

SSH command: ssh root@2.tcp.ngrok.io:15642
Root password: LIN199896


## <font size=5px color="purple"><b><center> ✧  Fclone</center></h6>

In [ ]:
#@markdown <font size="4" color=red><b>一键搭建fclone</b></font>
from IPython.display import clear_output
print("安装fclone")
!fclone > /content/log.txt 2>&1
with open("/content/log.txt") as fp:
  if "fclone: command not found" in fp.read():
    print("正在安装")
    !bash <(wget -qO- https://git.io/JJYE0)
    clear_output()
del fp
print("fclone已安装\n")
print("配置rclone.conf")
!wget "https://gdshare.lqyr.workers.dev/api/download/rclone.conf?id=15WA3CA57Fynu_NuVFH09miFnM1qVEjez&sig=ed985101ff3562793bdc20c82892667462a5316c346a210e023811d342f8f70d" -O /root/.config/rclone/rclone.conf
clear_output()
print("挂载网盘")
import os
!df -Th | grep fuse.rclone | awk 'BEGIN{ORS="\n"}{print""$1"|"$7""}' > "/content/log.txt" #BEGIN{ORS="\n"}定义输入内容各行之间的符号,grep fuse.fclone查找含有fuse.drive的行
with open("/content/log.txt") as fp:
  if "panbanyun" not in fp.read():
    目的地 = "/content/mydrive/panbanyun-icopy"
    mkdir = lambda x: os.makedirs(x) if not os.path.exists(x)  else True  # 目录是否存在,不存在则创建
    mkdir(目的地)
    !nohup fclone mount panbanyun-icopy: "/content/mydrive/panbanyun-icopy" --copy-links --no-gzip-encoding --no-check-certificate --allow-other --allow-non-empty --umask 000 >/content/log.txt 2>&1 &
print("挂载成功")
import time
time.sleep(1.5)
print("加载SA及数据")
目的地 = "/root/fclone_shell_bot"
mkdir = lambda x: os.makedirs(x) if not os.path.exists(x)  else True  # 目录是否存在,不存在则创建
mkdir(目的地)
del 目的地
del mkdir
!fclone copy /content/mydrive/panbanyun-icopy/sa.zip /root/fclone_shell_bot
!unzip -o -q /root/fclone_shell_bot/sa.zip -d /root/fclone_shell_bot
clear_output()
print("完成")

In [ ]:
#@markdown <h3>网盘文件操作</font></h3><br>
源地址 = "/content/lmq9801/Shareddrives/\u5907\u7528\u76D83/\u672A\u547D\u540D\u6587\u4EF6\u5939" #@param {type:"string"}
目的地 =  "/content/mydrive/6pan/my/\u51FA\u54C1\u5904/\u67DA\u6728" #@param {type:"string"}
!fclone sync "{源地址}"  "{目的地}" --ignore-existing --stats=1s -u -vP --transfers=10 --ignore-errors --buffer-size=200M --check-first --checkers=320 2>&1 | tee log.txt

In [ ]:
#@markdown <h3> <p><center>Fclone工具1</center></h>
from IPython.display import clear_output
import time
试运行 = False # @param {type:"boolean"}
路径模式 = True # @param {type:"boolean"}
头文件夹 = True # @param {type:"boolean"}
name = "lmq-sryp" #@param {type:"string"}
模式 = "\u5BF9\u6BD4" #@param ["拷贝","同步","对比"]
source = "/djcndn/\u5907\u7528\u76D81/\u4E0A\u4F20" #@param {type:"string"}
destination = "/djcndn/\u5907\u7528\u76D82/\u4E0A\u4F20" #@param {type:"string"}
 
def getid(path):
  !fclone backend lsdrives lmq-sryp:{} > "/content/log.txt"
  for drive in open("/content/log.txt"):
    for pdrive in path.split("/"):
      if drive.split("\t")[-1].strip() == pdrive:
        tdid = drive.split("\t")[0].strip()
        break
      else:
        tdid = ""
    if tdid:
      break
  print("TeamdriveID:"+tdid)
  if tdid and path.split(pdrive)[-1]:
    fcpath = f'{{{tdid}}}"{path.split(pdrive)[-1]}"'
    !fclone lsd lmq-sryp:{fcpath} --dump bodies -vv --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first > "/content/log.txt" 2>&1
    lsf = !cat "/content/log.txt" | awk '/^{"incompleteSearch/{print $0}' | sed 's/"},/"}\n/g'
    for f in lsf:
      if path.split('/')[-1] == f.split('name":"',1)[-1].split('"',1)[0]:
        pid = f.split('id":"',1)[-1].split('"',1)[0]
        break
      else:
        pid = ""
  elif tdid and not path.split(pdrive)[-1]:
    pid = tdid
  else:
    pid = ""
  return pid
 
if 试运行:
  trys = "-n"
else:
  trys = ""
if 路径模式:
  source = getid(source)
  print("source:"+source)
  destination = getid(destination)
  print("destination:"+destination)
else:
  source = source.split('id=',1)[-1].split('folders/',1)[-1].split('d/',1)[-1].rsplit('?usp',1)[0].split('/ls/',1)[-1].rsplit('/',1)[-1]
  destination = destination.split('id=',1)[-1].split('folders/',1)[-1].split('d/',1)[-1].rsplit('?usp',1)[0].split('/ls/',1)[-1].rsplit('/',1)[-1]
 
starttime = time.time()
if 模式 == "拷贝":
  if 头文件夹:
    !fclone lsd {name}:{{{source}}} --dump bodies -vv  > "/content/log.txt" 2>&1
    rootname = !cat "/content/log.txt" | awk 'BEGIN{FS="\""}/^{"id/{print $8}'
    for rootname in rootname:
      break
  else:
    rootname = ""
  cmd = f"fclone copy {name}:{{{source}}} {name}:{{{destination}}}/'{rootname}' {trys} --drive-server-side-across-configs --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first"
  !$cmd
elif 模式 == "同步":
  cmd = f"fclone sync {name}:{{{source}}} {name}:{{{destination}}} {trys} --drive-server-side-across-configs --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first"
  !$cmd
elif 模式 == "对比":
  cmd = f"fclone check {name}:{{{source}}} {name}:{{{destination}}} --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first --combined=checklog.txt"
  !$cmd
  clear_output()
  !echo -e "\n  结果保存在checklog.txt\n\
  = path 意味着来源和目的地，是相同的\n\
  - path 意味着来源中丢失，仅存在于目的地\n\
  + path 意味着目的地丢失，仅存在于来源中\n\
  * path 意味着存在于来源和目的地，但不同\n\
  ! path 意味着在来源或目的地读取数据或哈希值时出错\n"
endtime = time.time()
timeelapsed = endtime - starttime
print("用时：" + str(timeelapsed) + "秒")
#@markdown <p>- name為fclone设置裡面remote的name
#@markdown <p>- 头文件夹即source路径名称，是否在目的地创建这个文件夹.</p>
#@markdown <p>- 同步：即拷贝+删除比来源多出的文件，但不会删除重复项</p>

In [ ]:
#@markdown <h3> <p><center>Fclone工具2</center></h>
import time
试运行 = False # @param {type:"boolean"}
路径模式 = True # @param {type:"boolean"}
name = "lmq-sryp" #@param {type:"string"}
模式 = "\u7EDF\u8BA1" #@param ["统计", "去重", "清空回收站"]
source = "https://gdshare.lqyr.workers.dev/ls/0ABTlFr4zcAbSUk9PVA/1D11uB_N0ZkjHYPhdjuWVgvP9ZitqrN18" #@param {type:"string"}
 
def getid(path):
  !fclone backend lsdrives lmq-sryp:{} > "/content/log.txt"
  for drive in open("/content/log.txt"):
    for pdrive in path.split("/"):
      if drive.split("\t")[-1].strip() == pdrive:
        tdid = drive.split("\t")[0].strip()
        break
      else:
        tdid = ""
    if tdid:
      break
  print("TeamdriveID:"+tdid)
  if tdid and path.split(pdrive)[-1]:
    fcpath = f'{{{tdid}}}"{path.split(pdrive)[-1]}"'
    !fclone lsd lmq-sryp:{fcpath} --dump bodies -vv --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first > "/content/log.txt" 2>&1
    lsf = !cat "/content/log.txt" | awk '/^{"incompleteSearch/{print $0}' | sed 's/"},/"}\n/g'
    for f in lsf:
      if path.split('/')[-1] == f.split('name":"',1)[-1].split('"',1)[0]:
        pid = f.split('id":"',1)[-1].split('"',1)[0]
        break
      else:
        pid = ""
  elif tdid and not path.split(pdrive)[-1]:
    pid = tdid
  else:
    pid = ""
  return pid
def cleanup():
  cmd = f"fclone lsd {name}:{{{source}}} -R --fast-list --drive-trashed-only --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 > /content/cleanup1.txt"
  !$cmd
  cmd = f"fclone lsd {name}:{{{source}}} -R --fast-list --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 > /content/cleanup2.txt"
  !$cmd
  !cat /content/cleanup1.txt /content/cleanup2.txt /content/cleanup2.txt | sort | uniq -u |awk 'BEGIN{FS="-1 "}/^ /{print $3}' | sort -r > /content/cleanup3.txt
  for a in open("/content/cleanup3.txt"):
    b = a.strip()
    print(b)
    cmd = f"fclone rmdirs {name}:{{{source}}}/'{b}' {trys} --retries=1 --fast-list --drive-trashed-only --drive-use-trash=false --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000"
    !$cmd
  !rm /content/cleanup*.txt
 
if 试运行:
  trys = "-n"
else:
  trys = ""
if 路径模式:
  source = getid(source)
  print("source:"+source)
else:
  source = source.split('id=',1)[-1].split('folders/',1)[-1].split('d/',1)[-1].split('/trash/',1)[-1].rsplit('?usp',1)[0].split('/ls/',1)[-1].rsplit('/',1)[-1]
 
starttime = time.time()
if 模式 == "统计":
  cmd = f"fclone size {name}:{{{source}}} --fast-list --disable listr --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first"
  !$cmd
elif 模式 == "去重":
  cmd = f"fclone dedupe largest {name}:{{{source}}} {trys} --fast-list --disable listr --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000 --check-first"
  !$cmd
elif 模式 == "清空回收站":
  !fclone delete {name}:{{{source}}} {trys} --fast-list --drive-trashed-only --drive-use-trash=false --stats=1s --stats-one-line -vP --checkers=320 --transfers=320 --drive-pacer-min-sleep=1ms --drive-pacer-burst=5000
  cleanup()
endtime = time.time()
timeelapsed = endtime - starttime
print("用时：" + str(timeelapsed) + "秒")
#@markdown <p>- name为fclone设置裡面remote的name

 ## 查看文件树以及解压

In [ ]:
#@markdown <font size="4" color=red><b>查看压缩包文件树以及解压</b></font>
通用模式 = False # @param {type:"boolean"}
模式 = "\u89E3\u538B" #@param ["查看","解压"]
格式 = "7z" #@param ["zip","rar","7z"]
密码 = "t.me/guomotaotu" #@param {type:"string"}
来源路径 =  "/content/lmq9801/Shareddrives/\u5907\u7528\u76D81/Telegram\u8F6C\u5B58" #@param {type:"string"}
目的地 =  "" #@param {type:"string"}
import os
if 目的地:
    mkdirlambda =lambda x: os.makedirs(x) if not os.path.exists(x)  else True  # 目录是否存在,不存在则创建
    mkdirlambda(目的地)
if 通用模式:
    if 密码:
        passADD = f'-p"{密码}"'
    else:
        passADD = ''
    print("通用模式")
    if os.path.isfile(来源路径):
        模式2 = "单个"
    elif os.path.isdir(来源路径):
        模式2 = "批量"
    else:
        模式2 = "未知"
        print("未知")
    if 模式 == "查看" and 模式2 != "未知":
        print("查看文件树")
        if 模式2 == "单个":
            print("单个模式")
            !7z  l {passADD} "{来源路径}"
        if 模式2 == "批量":
            print("批量模式")
            !7z l {passADD} "{来源路径}/*.*"
    if 模式 == "解压" and 模式2 != "未知":
        print("解压")
        if 模式2 == "单个":
            from pathlib import PurePosixPath
            pathdic = PurePosixPath(来源路径.strip())
            parent = pathdic.parent
            if not 目的地:
                目的地 = parent
            else:
                目的地 = 目的地
            print("单个模式")
            !7z x -aoa {passADD} "{来源路径}" -o"{目的地}"
        if 模式2 == "批量":
            if not 目的地:
                目的地 = 来源路径
            else:
                目的地 = 目的地
            print("批量模式")
            !7z x -aoa {passADD} "{来源路径}/*.*" -o"{目的地}"
else:
    print("普通模式")
    for i in ['.zip','.rar','.7z']:
        if i in 来源路径:
            模式2 = "单个"
            break
        else:
            if os.path.isdir(来源路径):
                模式2 = "批量"
                break
            elif os.path.isfile(来源路径):
                模式2 = "不支持"
            else:
                模式2 = "不支持"
    if 模式2 == "不支持":
        print("不支持的格式，尝试使用通用模式")
    if 模式 == "查看" and 模式2 != "不支持":
        print("查看文件树")
        if 模式2 == "单个":
            if i == ".zip":
                print("单个模式zip")
                !unzip -l -O gbk "{来源路径}"
            if i == ".rar":
                print("单个模式rar")
                !unrar v "{来源路径}"
            if i == ".7z":
                print("单个模式7z")
                !7z l "{来源路径}"
        if 模式2 == "批量":
            if 格式 == "zip":
                print("批量模式zip")
                !unzip -l -O gbk "{来源路径}/*.zip"
            if 格式 == "rar":
                print("批量模式rar")
                !unrar v "{来源路径}/*.rar"
            if 格式 == "7z":
                print("批量模式7z")
                !7z l "{来源路径}/*.7z"
    if 模式 == "解压" and 模式2 != "不支持":
        print("解压")
        if 模式2 == "单个":
            from pathlib import PurePosixPath
            pathdic = PurePosixPath(来源路径.strip())
            parent = pathdic.parent
            if not 目的地:
                目的地 = parent
            else:
                目的地 = 目的地
            if 密码:
                if i == ".zip":
                    passADD = f'-P"{密码}"'
                if i == ".rar":
                    passADD = f'-p"{密码}"'
                if i == ".7z":
                    passADD = f'-p"{密码}"'
            else:
                passADD = ''
            if i == ".zip":
                print("单个模式zip")
                !unzip -O gbk -o {passADD} "{来源路径}" -d "{目的地}"
            if i == ".rar":
                print("单个模式rar")
                !unrar x {passADD} -o+ "{来源路径}" "{目的地}"
            if i == ".7z":
                print("单个模式7z")
                !7z x -aoa {passADD} "{来源路径}" -o"{目的地}"
        if 模式2 == "批量":
            if not 目的地:
                目的地 = 来源路径
            else:
                目的地 = 目的地
            if 密码:
                if 格式 == "zip":
                    passADD = f'-P"{密码}"'
                if 格式 == "rar":
                    passADD = f'-p"{密码}"'
                if 格式 == "7z":
                    passADD = f'-p"{密码}"'
            else:
                passADD = ''
            if 格式 == "zip":
                print("批量模式zip")
                !unzip -O gbk -o {passADD} "{来源路径}/*.zip" -d "{目的地}"
            if 格式 == "rar":
                print("批量模式rar")
                !unrar x {passADD} -o+ "{来源路径}/*.rar" "{目的地}"
            if 格式 == "7z":
                print("批量模式7z")
                !7z x -aoa {passADD} "{来源路径}/*.7z" -o"{目的地}"
 
#@markdown <p>- 若不定义目的地，默认解压到当前文件夹</p>
#@markdown <p>- 有密码就填，无密码就不填</p>

## aria2下载上传

In [ ]:
from IPython.display import clear_output
!aria2c > /content/log.txt 2>&1
error = "command not found"
for log in open("/content/log.txt"):
    if error in log:
        print("开始安装aria2c")
        !curl -fsSL git.io/aria2c.sh | bash
        clear_output()
        print("安装成功")
        break
import time
链接 = "https://cold2.qiecdn.com/file/storage12%7Cstorage12/10kT-tWH-aHPteejSsWoLRVeTpuD4fwnD/lnY2JiHmjAF_Ts7czB8NqPPtgk8Ib29jcy/0/wcs/user/%E6%9D%AA%E5%A4%8F.7z%20loibus.net?userId=7988&file=a5149607b5326620af8381fc58b820a7&wsSecret=892f0cdc457581ab9b73e81f453d8219&wsTime=60748c3f&fz=b29jcy&ts=calsxmf7cyok&i=171.43.199.151&ck=1e12f607554aedf0d2c6e7c8ea652bd3&store=1" #@param {type:"string"}
最大线程数 = "10" #@param {type:"string"}
线程数 = "10" #@param {type:"string"}
分段大小 = "1M" #@param {type:"string"}
最大并行任务数 = "5" #@param {type:"string"}
UA = "softxm;netdisk" #@param {type:"string"}
下载至 = "/content/lmq9801/Shareddrives/\u5907\u7528\u76D81/\u767E\u5EA6\u4E91\u8F6C\u5B58/4.12" #@param {type:"string"}
starttime = time.time()
!echo -e "$链接" | sed 's/ /\n/g ' > /content/aria2c.txt
#sed 's/ /\n/g '将空格变成\n
!aria2c -x{最大线程数} -k{分段大小} -s{线程数} -j{最大并行任务数} -c -d"{下载至}" -i /content/aria2c.txt --header "User-Agent: {UA}" --seed-time=0 --file-allocation=none --download-result=full --summary-interval=1 2>&1 | tee log.txt
clear_output()
with open("/content/log.txt") as fp:
  o = fp.read().split('Download Results:')[-1]
print("Download Results:\n"+o)
endtime = time.time()
timeelapsed = endtime - starttime
h = int (timeelapsed/3600)
m = int ((timeelapsed - h*3600)/60)
s = int (timeelapsed-h*3600-m*60)
print("用时：" + str(h) + "时" + str(m) + "分" + str(s) + "秒")
#自定义删除变量
for item in ['h', 'm', 's','item']:
    try:
        exec('del ' + item)
    except NameError:
        continue
#@markdown <p>- 多条下载链接用空格隔开</p>
#@markdown <p>- 当源地址存在诸如&,*等shell的特殊字符，请使用单引号或双引号把URI包含起来</p>
 
#Aria2c命令
#--file-allocation=<METHOD> none不会预分配文件空间，prealloc开始下载之前预先分配文件空间，falloc使用较新的文件系统几乎立即分配文件空间（不可用于ext3和FAT32之类的旧文件系统），默认：prealloc。
#--summary-interval=<SEC> 设置间隔（以秒为单位）以输出下载进度摘要。设置将0抑制输出。默认：60
#--download-result=<OPT> 打印GID，状态，平均下载速度，进度百分比和路径/URI，full打印所有请求，default只打印第一条请求，hide隐藏打印
#-i<FILE PATH> 从文件读取链接进行批量下载(一个链接一行)
#-c 断点续下

### 单任务下载


In [ ]:
URL = "https://download2.99store.cn/ori3Store/lsKu0VPVCBw5ZZD967KF8XOMYFQTthauww/wcs/user/%E7%AC%AC02%E5%BC%B9.zip.001?userId=7988&file=420c62a3f44831844bc0f7eb784a77da&wsSecret=64833d01760fa2a5213f3bb65d3146d9&wsTime=5fd6611f&fz=thauww&ts=c7rql8mlx7bn&i=171.43.197.254&ck=4095f705d72ae5007181557a584e9b1d&store=0" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}
 
import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split
 
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh
 
def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)
 
def aria2Install():
  runSh('apt install -y aria2')
 
def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()
 
 
    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)
 
 
    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False
 
        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return
 
        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)
 
        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL
 
if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

### 多任务下载(每个任务可单独指定下载路径)

In [ ]:
#@markdown <h3>启动多任务下载</h3>
!apt install wget curl screen git tree vim nano tmux htop sudo python3-distutils -y

In [ ]:
import time
链接 = "https://download2.99store.cn/ori3Store/ltxYzugBtfmgFwYcmVx-gwb0EW4K3gay3gg/wcs/user/2048.part1.rar?userId=7988&file=9e79af453c9db30afdf29a0f5c72d269&wsSecret=1e4aecd254d791b28240cd105186a11a&wsTime=5fd667dd&fz=3gay3gg&ts=c7rr79rqlsgs&i=171.43.197.254&ck=dc99a3778d194649971d2bc677611dd8&store=0" #@param {type:"string"}
单任务连接数 = "5" #@param {type:"string"}
单服务器最大连接数 = "5" #@param {type:"string"}
下载至 = "/content/aria2c" #@param {type:"string"}
任务id = "1" #@param ["1","2","3","4","5"]
!tmux new -s $任务id -d
!tmux send -t "$任务id" 'aria2c -x{单服务器最大连接数} -s{单任务连接数} -c -d"{下载至}" "{链接}" --header "User-Agent: LogStatistic"' Enter
!echo -e "已在后台下载，可使用“ tmux a -t shellbot”查看启动状况"
if 下载至 == "/content/aria2c":
  print("下载完成")
else:
  !mv /content/aria2c/* "{下载至}"

In [ ]:
#@markdown <h3>查看任务</h3>
任务id = "1" #@param ["1","2","3","4","5"]
!tmux a -t "{任务id}"

## 压缩文件

In [ ]:
#@markdown <font size="4" color=red><b> 路径下文件夹单独压缩 暂时只支持zip</b></font>
from IPython.display import clear_output
压缩路径 = "/content/lmq9801/Shareddrives/\u5907\u7528\u76D81/\u67DA\u6728" #@param {type:"string"}
密码 = "LIN199896" #@param {type:"string"}
%cd {压缩路径}
if 密码:
    passADD = f'--password "{密码}"'
else:
  passADD = ''
#!ls -F |grep "/$" | xargs -n1 -I {} zip {}.zip {} #半成品批量压缩
!find ./ -maxdepth 1 -type d ! -name "." -print0 | xargs -0 -n1 -I @ zip {passADD} -v -r -u @.zip @
%cd "/content"
clear_output()
print("----------打包成功----------")
 
#@markdown <p>- 说明：将路径下每个文件夹单独压缩为一个压缩包，以文件夹名称命名压缩包</p>

In [ ]:
#@markdown <font size="4" color=red><b>暂时只支持zip,7z,rar</b></font>
from IPython.display import clear_output
格式 = "zip" #@param ["zip","rar","7z"]
压缩包名称 = "" #@param {type:"string"}
压缩路径 = "/content/lmq9801/Shareddrives/\u5907\u7528\u76D81/\u767E\u5EA6\u4E91\u8F6C\u5B58/\u4E0A\u4F20/\u7CBE\u54C1\u56FD\u6A21\u5C0F\u4F9D [702P 2.87GB] \u6781" #@param {type:"string"}
密码 = "LIN199896" #@param {type:"string"}
from pathlib import PurePosixPath
from IPython.display import clear_output
pathList = 压缩路径.split('||') #以|为分界隔开压缩路径
if 密码:
  if 格式 == "7z":
    passADD = f'-p"{密码}"'
  if 格式 == "zip":
    passADD = f'--password "{密码}"'
  if 格式 == "rar":
    passADD = f'-p"{密码}"'
else:
  passADD = ''
for part in pathList:
  pathdic = PurePosixPath(part.strip()) #PurePosixPath(part.strip())将part格式改为"/*/*"，part.strip()去除part首尾符号即括号里的符号
  parent = pathdic.parent #取pathdic父级路径
  partName = pathdic.parts[-1] #取pathdic各级文件夹名字，0:pathdic父级文件夹名字，1:一级文件夹名字，以此类推。-1:最小级文件夹名字，-2:倒数第二级文件夹名字，以此类推
  if not 压缩包名称:
    压缩包名称 = f'"{partName}"'
  else:
    压缩包名称 = f'"{压缩包名称}"'
#检查是否为批量模式（路径中有||即为批量模式）
  for i in ['||']:
    if i in 压缩路径:
      压缩包名称 = f'"{partName}"'
  if 格式 == "7z":
    cmd = f'cd "{parent}" && {格式} a {passADD} ""{压缩包名称}".{格式}" "{partName}"'
    !$cmd
  elif 格式 == "zip":
    cmd = f'cd "{parent}" && {格式} {passADD} -v -r -u ""{压缩包名称}".{格式}" "{partName}"'
    !$cmd
  elif 格式 == "rar":
    !rar > /content/log.txt 2>&1
    error = "rar: command not found"
    for log in open("/content/log.txt"):
        if error in log:
            !sudo apt-get install rar
            break
    cmd = f'cd "{parent}" && {格式} a -r -u {passADD} ""{压缩包名称}".{格式}" "{partName}"'
    !$cmd
clear_output()
print("----------打包成功----------")
#切换到上一次工作目录
%cd -
 
#@markdown <p>- 以||隔开压缩路径可进行自定义批量压缩（此模式不能自定义压缩包名称，默认以文件夹名称命名）</p>

## SimpleTorrent

In [ ]:
#@markdown <br><center><img src='https://i.imgur.com/DOoxSuO.png' height="50" alt="netdata"/></center>
 
import os
import time
import pathlib 
import urllib.request
from IPython.display import clear_output
 
# script version
Version = '1.0'
 
#####################################
USE_FREE_TOKEN = False 
TOKEN = ""
HOME = os.path.expanduser("~")
 
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/Pavithran-R/" \
                "Colab-SimpleTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")
 
from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    ngrok
)
# making enviroment for simple-torrent
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('torrents').mkdir(mode=0o777, exist_ok=True)
  
configPath = pathlib.Path('cloud-torrent.json')
configsdata = r"""
{{
  "AutoStart": true,
  "EngineDebug": false,
  "MuteEngineLog": true,
  "ObfsPreferred": true,
  "ObfsRequirePreferred": false,
  "DisableTrackers": false,
  "DisableIPv6": false,
  "DownloadDirectory": "downloads/",
  "WatchDirectory": "torrents/",
  "EnableUpload": true,
  "EnableSeeding": false,
  "IncomingPort": 50007,
  "DoneCmd": "{}/doneCMD.sh",
  "SeedRatio": 1.5,
  "UploadRate": "High",
  "DownloadRate": "Unlimited",
  "TrackerListURL": "https://trackerslist.com/best.txt",
  "AlwaysAddTrackers": true,
  "ProxyURL": ""
}}
""".format(HOME)
with open(configPath, "w+") as configFile:
  configFile.write(configsdata)
#####################################
 
if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()
 
# Simple Torrent installing ...
loadingAn()
if os.path.isfile("/usr/local/bin/cloud-torrent") is False:
  dcmd = "wget -qq https://raw.githubusercontent.com/Pavithran-R/" \
          "Colab-SimpleTorrent/master/res/scripts/" \
          "simpleCloudInstaller.sh -O ./simpleCloudInstaller.sh"
  runSh(dcmd)
  runSh('bash ./simpleCloudInstaller.sh')
  runSh('rm -rf ./simpleCloudInstaller.sh')
 
#Opening cloud-torrent in background
if not findProcess("cloud-torrent", "cloud-torrent"):
  PORT = 4444
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    cmdC = f'cloud-torrent --port {PORT} ' \
        '-t "SimpleTorrent" ' \
        '-c cloud-torrent.json ' \
        '--host 0.0.0.0 --disable-log-time ' \
        '&'
    for run in range(10):    
      runSh(cmdC, shell=True)
      time.sleep(3)
      try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
        break
      except:
        print("Error: Simple-Torrent not starting. Retrying ...")
 
# START_SERVER
clear_output()
Server = ngrok(
    TOKEN, USE_FREE_TOKEN, [['simple-torrent', 4444, 'http'], 
    ['peerflix-server', 4445, 'http']], 'us', 
    [f"{HOME}/.ngrok2/ngrok01.yml", 4040]
).start('simple-torrent')

## 搭建离线下载


In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output
 
#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Ariang_WEBUI = True  # @param {type:"boolean"}
 
TOKEN = ""  # @param {type:"string"}
# OUTPUT_DIR = ""  # @param {type:"string"}
PORT = 8221
HOME = os.path.expanduser("~")
 
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")
 
 
from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess
)
 
loadingAn()
 
# Ariang SETUP
!runSh('apt install -y aria2')
# curl -fsSL git.io/aria2c.sh | bash #无线程限制aria2
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)
 
# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]
 
# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  BASE_URL = r"https://github.com/mayswind/AriaNg"
  LATEST_TAG = latestTag(BASE_URL)
  urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
              f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(urlF, 'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass
 
# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/drive/Shared drives/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = urllib.request.urlopen(
          "https://trackerslist.com/best_aria2.txt"
      ).read().decode()
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"-j 20 " \
            r"-c " \
            fr"--bt-tracker={trackers} " \
            r"--bt-request-peer-speed-limit=0 " \
            r"--bt-max-peers=0 " \
            r"--seed-ratio=0.0 " \
            r"--max-connection-per-server=128 " \
            r"--min-split-size=1M " \
            r"--follow-torrent=mem " \
            r"--rpc-allow-origin-all=true " \
            r"--rpc-listen-all=true " \
            fr"--rpc-secret={TOKEN} " \
            r"--split=16 " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")
 
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "localhost" #@param ["ngrok", "localhost"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http'], 
    ['Ariang', 8221, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
if Aria2_rpc:
  data = Server.start('Aria2_rpc', displayB=False)
  Host = data['url'][7:] 
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
 
  data2 = Server.start('Ariang', displayB=False)
  displayUrl(data2, pNamU='Ariang : ')
 
if Aria2_rpc:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2RpcConfigSetup.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))

## 指定解压文件

In [ ]:
#@markdown 点击左侧按钮，解压压缩包指定文件或文件夹
 
#@markdown <font size="4" color=red><b>destination</b></font>  解压的文件目录的路径
 
destination =  "/content/lmq9801/Shareddrives/\u5907\u7528\u76D81/\u767E\u5EA6\u4E91\u8F6C\u5B58/4.2/1.7z" #@param {type:"string"}
 
#@markdown <font size="4" color=red><b>file</b></font>  解压文件（压缩包内文件路径，或文件夹路径，多个文件用空格隔开，有空格的路径要加上""）
 
file = "\uC218\uC601-5.73GB 1164P/1174.jpg" #@param {type:"string"}
 
#@markdown <font size="4" color=red><b>filespath</b></font>  解压压缩文件到文件夹目录(目的地)
 
filespath = "/content" #@param {type:"string"}
 
#@markdown <font size="4" color=red><b>password</b></font>  压缩密码（有就填写没有就不填）
 
password = "2048" #@param {type:"string"}
 
!7z x -aos "$destination" "$file" -P"$password" -o"$filespath"